# データの可視化とPlotly

- [Matplotlib](https://matplotlib.org/)
- [seaborn](https://seaborn.pydata.org/)
- [Bokeh](https://bokeh.org/)
- [Plotly.py](https://plotly.com/python/)
- [Plotly Express](https://plotly.com/python/plotly-express/)

## Plotly.pyとPlotly Express


In [ ]:
import pandas as pd
plot_df = pd.DataFrame(
    data=[[4, 2], [1, 3]], 
    index=pd.Index(["前期", "当期"], name="期間"), 
    columns=pd.Index(["東京", "大阪"], name="地域")
)
plot_df

In [ ]:
import plotly.graph_objects as go
trace1 = go.Bar(x=plot_df.index, y=plot_df["東京"], name="東京")
trace2 = go.Bar(x=plot_df.index, y=plot_df["大阪"], name="大阪")
go_fig = go.Figure([trace1, trace2], layout=go.Layout(barmode="stack"))
go_fig.show()

In [ ]:
tidy_df = pd.melt(
    plot_df.reset_index(), 
    id_vars=["期間"], 
    value_vars=["東京", "大阪"], 
    value_name="金額"
)
tidy_df

In [ ]:
import plotly.express as px
ex_fig = px.bar(tidy_df, x="期間", y="金額", color="地域")
ex_fig.show()

In [ ]:
ex_fig.write_html("bar_graph.html")

In [ ]:
#!pip install kaleido==0.2.1

In [ ]:
#ex_fig.write_image("bar_graph.png")

## 構成割合の可視化


In [ ]:
composition_df = pd.DataFrame(
    [
        ["必要生活費", "衣類", 20],
        ["必要生活費", "食費", 30],
        ["必要生活費", "住居費", 40],
        ["教養娯楽費", "教養", 15],
        ["教養娯楽費", "趣味娯楽", 20],
    ],
    columns=["大分類", "中分類", "金額"]
)

### 円グラフとサンバーストグラフ


In [ ]:
pie_fig = px.pie(composition_df, names="中分類", values="金額")
pie_fig.update_traces(textinfo="label+percent", textfont_size=20, showlegend=False)
pie_fig.show()

In [ ]:
sunburst_fig = px.sunburst(composition_df, path=["大分類", "中分類"], values="金額")
sunburst_fig.update_traces(textinfo="label+value", textfont_size=20)
sunburst_fig.show()

### ツリーマップ


In [ ]:
treemap_fig = px.treemap(composition_df, path=["大分類", "中分類"], values="金額")
treemap_fig.update_traces(textinfo="label+value", textfont_size=20)
treemap_fig.show()

## 量の比較と推移の可視化


In [ ]:
ts_df = pd.DataFrame(
    [
        [2020, "必要生活費", "衣類", 20],
        [2020, "必要生活費", "食費", 30],
        [2020, "必要生活費", "住居費", 35],
        [2020, "教養娯楽費", "教養", 15],
        [2020, "教養娯楽費", "趣味娯楽", 15],
        [2021, "必要生活費", "衣類", 20],
        [2021, "必要生活費", "食費", 40],
        [2021, "必要生活費", "住居費", 35],
        [2021, "教養娯楽費", "教養", 10],
        [2021, "教養娯楽費", "趣味娯楽", 25],
        [2022, "必要生活費", "衣類", 15],
        [2022, "必要生活費", "食費", 25],
        [2022, "必要生活費", "住居費", 40],
        [2022, "教養娯楽費", "教養", 15],
        [2022, "教養娯楽費", "趣味娯楽", 30],
    ],
    columns=["年", "大分類", "中分類", "金額"]
).assign(**{"日付": lambda df: pd.to_datetime(df["年"], format="%Y")})

### 棒グラフ


In [ ]:
bar_fig = px.bar(ts_df, x="年", y="金額", color="中分類")
bar_fig.update_layout(xaxis={"dtick": 1})
bar_fig.show()

In [ ]:
groupbar_fig = px.bar(ts_df, x="日付", y="金額", color="大分類", barmode="group")
groupbar_fig.update_layout(xaxis={"dtick": "M12", "tickformat": "%Y", "ticksuffix": "年"})
groupbar_fig.show()

### 折れ線グラフ


In [ ]:
line_fig = px.line(ts_df, x="日付", y="金額", color="中分類", markers=True)
line_fig.show()

## 構成割合の推移の可視化

### エリアチャート


In [ ]:
area_fig = px.area(ts_df, x="日付", y="金額", color="中分類")
area_fig.show()

### その他の構成割合の推移の可視化

## レイアウトとグラフの組み合わせ

### Plotly Expressのカラースケール


In [ ]:
px.colors.qualitative.Plotly

In [ ]:
color_fig = px.colors.qualitative.swatches()
color_fig.show()

In [ ]:
px.colors.qualitative.Alphabet

### グラフの組み合わせ


In [ ]:
fig2 = go.Figure()
for cat in ts_df["大分類"].unique():
    tmp_df = ts_df.loc[
        ts_df["大分類"]==cat, ["日付", "金額"]
    ].groupby("日付").sum().reset_index()
    # add_traceメソッドで各大分類ごとに描画領域に折れ線グラフを追加
    fig2.add_trace(go.Scatter(
        x=tmp_df["日付"], 
        y=tmp_df["金額"], 
        name=cat, 
        mode="lines+markers",  # 折れ線グラフにマーカー(点)を表示
        line={"width": 4, "dash": "dash"},  # 線の幅と形状を設定
        marker={"size": 15, "symbol": "diamond"},  # マーカーのサイズと形状を設定
        yaxis="y1",  # y軸(縦軸)の1軸目を使用
    ))
total_df = ts_df[["日付", "金額"]].groupby("日付").sum().reset_index()
# 各年の総計を棒グラフとして追加
fig2.add_trace(go.Bar(
    x=total_df["日付"], 
    y=total_df["金額"], 
    name="総計", 
    yaxis="y2",  # y軸(縦軸)の2軸目を使用
    opacity=0.6,  # 不透明度を0.6に設定
))
# レイアウトを更新
fig2.update_layout(
    width=1000,  # 描画領域の横幅を設定
    height=500,  # 描画領域の縦幅を設定
    font={"size": 14},  # フォントサイズを設定
    xaxis={
        "dtick": "M12",  # x軸(横軸)の目盛り間隔を12ヶ月に設定
        "tickformat": "%Y",  # x軸(横軸)の目盛りの表示形式を設定
        "ticksuffix": "年",  # x軸(横軸)の目盛りに単位を追加
        "title": None,  # x軸(横軸)のタイトルを非表示
    },
    yaxis={
        "title": "大分類の金額", 
        "side": "left",  # y軸(縦軸)の1軸目を左側に設定
    },
    yaxis2={
        "ticksuffix": "円",  # y軸(縦軸)の2軸目の目盛りに単位を追加
        "title": "総計の金額", 
        "side": "right",  # y軸(縦軸)の2軸目を右側に設定
        "overlaying": "y",  # y軸(縦軸)の2軸目をy軸(縦軸)の1軸目に重ねて表示  
    }, 
    legend={"x": 1.1},  # 凡例の位置を右寄りに設定
    legend_title_text="支出",  # 凡例のタイトルを設定
    title={
        "text": "年ごとの支出", 
        "x": 0.5,  # タイトルの位置を中央に設定
    },
) 
fig2.show()

### サブプロット


In [ ]:
from plotly.subplots import make_subplots
# 1行2列の描画領域を用意
subplot_fig = make_subplots(rows=1, cols=2, subplot_titles=("棒グラフ", "折れ線グラフ"))
# 棒グラフに設定するカラーと模様を用意
colors_pattern = px.colors.qualitative.Pastel1
shape_pattern = "/.+x\\"
# 棒グラフの追加
bar_data = ts_df.loc[ts_df["年"] == 2022, ["中分類", "金額"]]
for n, cat in enumerate(bar_data["中分類"]):
    tmp_df = bar_data[bar_data["中分類"]==cat]
    # add_traceメソッドで中分類ごとに色や模様の異なる棒グラフを追加
    subplot_fig.add_trace(
        go.Bar(
            x=tmp_df["中分類"], 
            y=tmp_df["金額"], 
            name=cat, 
            texttemplate="%{y}万円",  # texttemplateでy軸の金額を棒グラフ中に表示
            textfont_size=15,  # フォントサイズを設定
            textposition="inside",  # 棒グラフの内側に表示
            marker_color=colors_pattern[n],  # 棒グラフの色を設定
            marker_pattern_shape=shape_pattern[n],  # 棒グラフの模様を設定
        ), 
        row=1,  # 1行目の描画領域に表示
        col=1,  # 1列目の描画領域に表示
    )
# 折れ線グラフの追加
line_data = ts_df.pivot_table(index="日付", columns="大分類", values="金額", aggfunc="sum")
for col in line_data:
    # add_traceメソッドで大分類ごとに折れ線グラフを追加
    subplot_fig.add_trace(
        go.Scatter(
            x=line_data.index,
            y=line_data[col], 
            name=col,
            mode="lines+markers",  # 折れ線グラフにマーカー(点)を表示
        ),
        row=1,  # 1行目の描画領域に表示 
        col=2,  # 2列目の描画領域に表示
    )
# グラフのレイアウトを更新
subplot_fig.update_layout(
    width=1000, 
    height=400,
    legend={
        "orientation": "h",  # "h" は水平（horizontal）を意味し、凡例の項目が水平方向に並ぶ
        "x": 0.5,  # 0.5 はグラフの水平方向の中央
        "y": -0.5,  # -0.5 は、グラフ領域外の下部（グラフの下端からさらに10%下）に凡例を配置
        "xanchor": "center",  # 凡例の中央が x で指定された位置に配置
        "yanchor": "top",  # 凡例の上端が y で指定された位置に配置
    },
)
# x軸(横軸)の設定
subplot_fig.update_xaxes(
    tickformat="%Y/%m",  # x軸(横軸)の目盛りの表示形式を設定
    tickangle=45,  # x軸(横軸)の目盛りを45度傾けて表示
)
subplot_fig.show()

## 分布の可視化


In [ ]:
df = pd.DataFrame([
    ["いちごストア株式会社", "A4491", "上場", "小売業", 50, 250],
    ["メガハード株式会社", "A3547", "上場", "製造業", 50, 300],
    ["百聞半導体株式会社", "A2704", "上場", "製造業", 20, 180],
    ["五十音サーチ株式会社", "A8008", "上場", "小売業", 30, 200],
    ["利根川通販株式会社", "A4342", "上場", "小売業", 100, 240],
    ["超手本株式会社", "A3674", "上場", "サービス業", 20, 150],
    ["源内自動車株式会社", "A7514", "非上場", "製造業", 10, 100],
    ["クローズサピエンス合同会社", "A0941", "非上場", "サービス業", None, 120],
    ["富価自動車株式会社", "J7203", "上場", "製造業", 60, 120],
    ["ハード通信株式会社", "J9984", "上場", "サービス業", 15, 50],
    ["株式会社財前", "J3994", "非上場", "サービス業", None, 30]
    ], columns=["会社名", "会社コード", "上場区分", "業種", "従業員数", "資本金"]
)

### ヒストグラム


In [ ]:
hist_fig = px.histogram(df, x="資本金")
hist_fig.show()

### 箱ひげ図とバイオリン図


In [ ]:
box_fig = px.box(df, x="上場区分", y="資本金")
box_fig.show()

In [ ]:
violin_fig = px.violin(df, x="上場区分", y="資本金", box=True, points="all")
violin_fig.show()

## 関係性の可視化

### 散布図


In [ ]:
scatter_fig = px.scatter(df, x="資本金", y="従業員数", color="上場区分")
scatter_fig.show()

### ヒートマップ


In [ ]:
# 等間隔離散化で区分を用意
df = df.assign(**{
    "従業員数区分": pd.cut(df["従業員数"], bins=3),
    "資本金区分": pd.cut(df["資本金"], bins=3)
})
# 各区分の件数をクロス集計でカウント
cols = ["従業員数区分", "資本金区分"]
crosstab = df[cols].pivot_table(
    index="従業員数区分", 
    columns="資本金区分", 
    aggfunc="size", 
    observed=False
).sort_index(ascending=False)
crosstab

In [ ]:
im_fig = px.imshow(
    crosstab.values,
    x=[str(i) for i in crosstab.columns],  # 従業員数区分を文字列型に変換
    y=[str(i) for i in crosstab.index],  # 資本金区分を文字列型に変換
    labels={"x": crosstab.columns.name, "y": crosstab.index.name},
    text_auto=True  # 区分ごと件数の表示
)
im_fig.show()

### バブルチャート


In [ ]:
gapminder_data = px.data.gapminder()
gapminder_df = gapminder_data[gapminder_data["year"]==2007]
gapminder_df.head()

In [ ]:
bubble_fig = px.scatter(gapminder_df,
    x="gdpPercap",
    y="lifeExp",
    size="pop",
    color="continent",
    log_x=True,
    size_max=60
)
bubble_fig.show()

## 地図・金融等の特定分野の可視化
